In [89]:
import pandas as pd

In [90]:
rated_df=data=pd.read_csv(r"G:\Anand\project 3 data.csv",encoding="latin1")
rated_df.head(n=2)

,cpc,competition,keyword,no ofresults,position,search vol,URL,domain,response content,response status,title,content found,domain found,homepage,title found,url found
0,0.59,1.0,grnlippmuschelpulver,63000,1,1300,https://www.barf-alarm.de/shop/barf-nahrungser,www.barf-alarm.de,Wnschen Sie einen individuellen Rinderfleis...,200,Grnlippmuschelpulver-Das Schmerzlinderungsmit...,0.0,0.0,0,0.0,0.0
1,1.21,1.0,grnlippmuschelpulver pferd,159000,67,90,https://www.barf-alarm.de/shop/barf-nahrungser,www.barf-alarm.de,Wnschen Sie einen individuellen Rinderfleis...,200,Grnlippmuschelpulver-Das Schmerzlinderungsmit...,0.0,0.0,0,0.0,0.0


In [91]:
domains = list(rated_df.domain.unique())
keywords = list(rated_df.keyword.unique())
def get_domain_id(domain):
    return domains.index(domain)
def get_keyword_id(keyword):
    return keywords.index(keyword)
rated_df['domain_id'] = rated_df['domain'].apply(get_domain_id)
rated_df['keyword_id'] = rated_df['keyword'].apply(get_keyword_id)
rated_df

,cpc,competition,keyword,no ofresults,position,search vol,URL,domain,response content,response status,title,content found,domain found,homepage,title found,url found,domain_id,keyword_id
0,0.59,1.00,grnlippmuschelpulver,63000,1,1300,https://www.barf-alarm.de/shop/barf-nahrungser,www.barf-alarm.de,Wnschen Sie einen individuellen Rinderfleis...,200,Grnlippmuschelpulver-Das Schmerzlinderungsmit...,0.000000,0.000000,0,0.000000,0.0,0,0
1,1.21,1.00,grnlippmuschelpulver pferd,159000,67,90,https://www.barf-alarm.de/shop/barf-nahrungser,www.barf-alarm.de,Wnschen Sie einen individuellen Rinderfleis...,200,Grnlippmuschelpulver-Das Schmerzlinderungsmit...,0.000000,0.000000,0,0.000000,0.0,0,1
2,0.77,0.93,grnlippmuschel kapseln nebenwirkungen,23000,42,90,https://www.barf-alarm.de/shop/barf-nahrungser...,www.barf-alarm.de,Wnschen Sie einen individuellen Rinderfleis...,200,Grnlippmuschelpulver-Das Schmerzlinderungsmit...,0.000000,0.000000,0,0.000000,0.0,0,2
3,1.24,1.00,grnlippmuschelextrakt fr hunde,105000,45,210,https://www.barf-alarm.de/shop/barf-nahrungser...,www.barf-alarm.de,Wnschen Sie einen individuellen Rinderfleis...,200,Grnlippmuschelpulver-Das Schmerzlinderungsmit...,0.000000,0.000000,0,0.000000,0.0,0,3
4,1.04,1.00,grnlippmuschelextrakt hund,51000,44,390,https://www.barf-alarm.de/shop/barf-nahrungser...,www.barf-alarm.de,Wnschen Sie einen individuellen Rinderfleis...,200,Grnlippmuschelpulver-Das Schmerzlinderungsmit...,0.000000,0.000000,0,0.000000,0.0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,1.75,0.92,barf shop,8180000,2,5400,https://www.barfshop.de/,www.barfshop.de,\nKundenlogin \nKundenlogin\nKonto erstellen\n...,200,barfshop.de - Startseite,0.500000,1.000000,1,1.000000,0.0,1,61
63,1.54,0.88,barfer shop,173000,2,50,https://www.barfshop.de/,www.barfshop.de,\nKundenlogin \nKundenlogin\nKonto erstellen\n...,200,barfshop.de - Startseite,0.500000,0.500000,1,0.500000,0.0,1,62
64,1.31,0.74,barf shop hamburg,91000,14,480,https://www.barfshop.de/,www.barfshop.de,\nKundenlogin \nKundenlogin\nKonto erstellen\n...,200,barfshop.de - Startseite,0.333333,0.666667,1,0.666667,0.0,1,60
65,1.37,0.80,barfshops,56000,7,50,https://www.barfshop.de/,www.barfshop.de,\nKundenlogin \nKundenlogin\nKonto erstellen\n...,200,barfshop.de - Startseite,0.000000,0.000000,1,0.000000,0.0,1,63


In [92]:
# Define Features (which rows should the ml algorithm should consider for learning)
features = ["domain_id", 
            "content found", 
            "domain found",
            "title found", 
            "url found", 
            "keyword_id", 
            "cpc", 
            "no ofresults",
            "search vol"]
# Define Target (What should be predicted)
target = "position"

In [93]:
# Split in train and test data
train = rated_df.sample(frac=0.8)
test = rated_df.loc[~rated_df.index.isin(train.index)]
print ("Train rows: {}".format(len(train.index)))
print ("Test rows: {}".format(len(test.index)))

Train rows: 54
Test rows: 13


In [94]:
# Train Model with Data
from sklearn.preprocessing import StandardScaler  
from sklearn.model_selection import LeaveOneOut
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline

sc = StandardScaler()

mlc = MLPClassifier(activation = 'relu', random_state=1,nesterovs_momentum=True)
loo = LeaveOneOut()
pipe = make_pipeline(sc, mlc)

# Train the Model and check wich of the Parameters works best
parameters = {"mlpclassifier__hidden_layer_sizes": [(50,),(100,)],  
              "mlpclassifier__solver": ("sgd", "lbfgs"), 
              "mlpclassifier__max_iter": [100, 500, 1000], 
              "mlpclassifier__learning_rate_init":[0.001, 0.1]  }
MLPClassifierModel = GridSearchCV(pipe, parameters, n_jobs= None,cv = None)
MLPClassifierModel.fit(train[features], train[target])

# Save Model to file to used it later
#file = open("k_t_o_MLPClassifierModel_10_comp.pkl", 'wb')
#pickle.dump(MLPClassifierModel, file)
#file.close()

c:\users\dell\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
c:\users\dell\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)
c:\users\dell\appdata\local\programs\python\python37\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
c:\users\dell\appdata\local\programs\python\python37\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: Converge

GridSearchCV(cv=None, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('standardscaler',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('mlpclassifier',
                                        MLPClassifier(activation='relu',
                                                      alpha=0.0001,
                                                      batch_size='auto',
                                                      beta_1=0.9, beta_2=0.999,
                                                      early_stopping=False,
                                                      epsilon=1e-08,
                                                      hidden_layer_sizes=(100,),
                                                      le

In [95]:
# Define Domain to optimize & competitors (more competitors=better results)
main_domain = ['www.barf-alarm.de']
competitor_domains = ['barf-factory.de', 'www.barfers-wellfood.de',
                      'www.barfinfo.de', 'www.barfkoeln.de', 'www.barfshop-koeln.de',
                      'www.barfshop.de']
base_domains = main_domain + competitor_domains

rated_df[rated_df["domain"]== "main_domain"]


,cpc,competition,keyword,no ofresults,position,search vol,URL,domain,response content,response status,title,content found,domain found,homepage,title found,url found,domain_id,keyword_id


In [96]:
# Get Data to query the Model
id = 2 # just change ID to one of the row ids from above
data = [
    rated_df.ix[id]["domain_id"],
    rated_df.ix[id]["content found"],
    rated_df.ix[id]["domain found"],
    rated_df.ix[id]["title found"],
    rated_df.ix[id]["url found"],
    rated_df.ix[id]["keyword_id"],
    rated_df.ix[id]["cpc"],
    rated_df.ix[id]["no ofresults"],
    rated_df.ix[id]["search vol"],
]

c:\users\dell\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  after removing the cwd from sys.path.
c:\users\dell\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  """
c:\users\dell\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.

In [97]:
# What Data we query?
def print_data(data):
    print("domain: {}".format(domains[int(data[0])]))
    print("keyword: {}".format(keywords[int(data[5])]))
    print("keyword in content found? {}".format(data[1]))
    print("keyword in title found? {}".format(data[3]))
    print("keyword in domain Found? {}".format(data[2]))
    print("keyword in URL found? {}".format(data[4]))
    print("cpc: {}".format(data[6]))
    print("no ofresults: {}".format(data[7]))
    print("search vol: {}".format(data[8]))
print_data(data)

domain: www.barf-alarm.de
keyword: grnlippmuschel kapseln nebenwirkungen
keyword in content found? 0.0
keyword in title found? 0.0
keyword in domain Found? 0.0
keyword in URL found? 0.0
cpc: 0.77
no ofresults: 23000
search vol: 90


In [98]:
# Predict Position
df_to_predict = pd.DataFrame(data = [data], index=[0], columns=features)
res = MLPClassifierModel.predict(df_to_predict)
print("MLPClassifierModel predicted position:  {}".format(int(res[0])))


MLPClassifierModel predicted position:  55


In [99]:
# Modify Data for that Keyword
modified_data = data
data[1] = 1
data[3] = 63000
print_data(modified_data)

domain: www.barf-alarm.de
keyword: grnlippmuschel kapseln nebenwirkungen
keyword in content found? 1
keyword in title found? 63000
keyword in domain Found? 0.0
keyword in URL found? 0.0
cpc: 0.77
no ofresults: 23000
search vol: 90


In [100]:
# Predict Position
df_to_predict = pd.DataFrame(data = [modified_data], index=[0], columns=features)
res = MLPClassifierModel.predict(df_to_predict)
print("MLPClassifierModel predicted Position:  {}".format(int(res[0])))

MLPClassifierModel predicted Position:  22
